In [36]:
# importar os pacotes necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# importar o dataset para um dataframe
df = pd.read_csv('sao-paulo-properties-april-2019.csv')

# ver as 5 primeiras entradas
df.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525025,-46.482436


In [37]:
for setor in df.District.unique():
  df[setor] = 0
  df[df.District == setor][setor] = 1
df.drop('District', axis = 1, inplace = True)
df.drop(['Property Type'], axis = 1, inplace = True)

df.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(13640, 110)

In [38]:
Sale = df[df['Negotiation Type'] == 'sale'].drop('Negotiation Type', axis = 1)
Sale.head(1)

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,Latitude,Longitude,Artur Alvim/São Paulo,Belém/São Paulo,Cangaíba/São Paulo,Carrão/São Paulo,Cidade Líder/São Paulo,Cidade Tiradentes/São Paulo,Ermelino Matarazzo/São Paulo,Iguatemi/São Paulo,Itaim Paulista/São Paulo,Itaquera/São Paulo,Jardim Helena/São Paulo,José Bonifácio/São Paulo,Lajeado/São Paulo,Parque do Carmo/São Paulo,Penha/São Paulo,Ponte Rasa/São Paulo,Sapopemba/São Paulo,São Lucas/São Paulo,São Mateus/São Paulo,São Miguel/São Paulo,São Rafael/São Paulo,Tatuapé/São Paulo,Vila Curuçá/São Paulo,Vila Formosa/São Paulo,Vila Jacuí/São Paulo,Vila Matilde/São Paulo,Vila Prudente/São Paulo,...,Freguesia do Ó/São Paulo,Jaguaré/São Paulo,Jaraguá/São Paulo,Lapa/São Paulo,Limão/São Paulo,Perdizes/São Paulo,Pinheiros/São Paulo,Pirituba/São Paulo,Raposo Tavares/São Paulo,Rio Pequeno/São Paulo,Vila Leopoldina/São Paulo,Barra Funda/São Paulo,Bela Vista/São Paulo,Bom Retiro/São Paulo,Brás/São Paulo,Consolação/São Paulo,Liberdade/São Paulo,Pari/São Paulo,República/São Paulo,Santa Cecília/São Paulo,Sé/São Paulo,Casa Verde/São Paulo,Jaçanã/São Paulo,Mandaqui/São Paulo,Santana/São Paulo,Tremembé/São Paulo,Tucuruvi/São Paulo,Vila Guilherme/São Paulo,Vila Maria/São Paulo,Medeiros/São Paulo,Água Rasa/São Paulo,Aricanduva/São Paulo,Guaianazes/São Paulo,Jardim São Luis/São Paulo,Vila Madalena/São Paulo,Mooca/São Paulo,Brooklin/São Paulo,Vila Olimpia/São Paulo,Perus/São Paulo,São Domingos/São Paulo
4901,732600,1000,74,1,2,1,2,1,0,1,0,-23.552129,-46.692244,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
def tirar_outlier(df, lista):
  df_clean = df[lista]
  for col, dados in df_clean.iteritems():
    df_clean = df_clean[((df_clean[col] - df_clean[col].mean()) / df_clean[col].std()).abs() < 3]
    df_clean = df_clean[np.abs(df_clean[col]-df_clean[col].mean()) <= (3*df_clean[col].std())]

  valores_retirados = df.shape[0] - df_clean.shape[0]
  print('Foi retirado {} valores de {} ({:.2f})%'.format(valores_retirados, df.shape[0], valores_retirados/df.shape[0]*100))
  print('Ficamos com {} resultados'.format(df_clean.shape[0]))

  for i in range(0, len(df)):
    if i not in df_clean.index:
        df = df[df.index != i]

  df = df.reset_index(drop=True)
  return df

lista = ['Latitude', 'Longitude', 'Price']

print('\nPara o Data Frame Sale:')
Sale_clean = tirar_outlier(Sale, lista)


Para o Data Frame Sale:
Foi retirado 739 valores de 6412 (11.53)%
Ficamos com 5673 resultados


In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(Sale_clean[['Price']])
Sale_clean['Price'] = scaler.transform(Sale_clean[['Price']])

In [41]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

X = Sale_clean.drop(['Price'], axis = 1)
y = Sale_clean[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

modelo = XGBRegressor(random_state=42, silent=True)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)
resultados = r2_score(y_test, y_pred)

#print('R² = {:.3f}'.format(r2_score(y_test, y_pred)))
resultados.round(3)

0.901

In [0]:
import pickle

with open('modelo_simples.pkl', 'wb') as file:
    pickle.dump(modelo, file)

In [0]:
features_simples = X_train.columns.values

with open('features_simples.names', 'wb') as file:
    pickle.dump(features_simples, file)

In [0]:
with open('/content/modelo_simples.pkl', 'rb') as file:
    modelo_simples = pickle.load(file)
with open('/content/features_simples.names', 'rb') as file:
    features_names = pickle.load(file)